In [1]:
import sys
sys.path.append("../")
import transformers
transformers.logging.set_verbosity_error()

from functools import partial
from transformers import AutoTokenizer, AutoModelForMaskedLM
from src.data.dataio import DataFiles, Dataset, remove_empty_fn, truncate_fn
from src.models.tokenizing import Vocabulary, Tokenizer

In [6]:
# data_files = DataFiles.from_url_file(url_file="../data/books.txt") // From URL
data_files = DataFiles.from_dir("/content/comp-550-project/data/articles/clean_articles/validation") # From dir

dataset = Dataset(data_files)
dataset = dataset.map(remove_empty_fn, batched=True)
vocab = Vocabulary(dataset["text"])
tokenizer = Tokenizer(vocab=vocab)
dataset = dataset.map(
    partial(truncate_fn, tokenizer=tokenizer, max_seq_length=24, fill_to_max=True), batched=True)

Resolving data files:   0%|          | 0/225 [00:00<?, ?it/s]

Using custom data configuration default-21248a72b27f72f6


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-21248a72b27f72f6/0.0.0/e10c27cd9493b3e5d594862e2a0d24e29034e10b895b72057984ed60b5a1c333. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

['mallon', 'wades', 'into', 'ne', 'vote', 'battle', 'middlesbrough', 'mayor', 'ray', 'mallon', 'has', 'been', 'drafted', 'in', 'to', 'boost', 'the', 'yes', 'campaign', 'as', 'the', 'north']
['east', 'assembly', 'referendum', 'enters', 'its', 'final', 'week', '.', 'the', 'former', 'police', 'chief', ',', 'dubbed', 'robocop', 'for', 'his', 'zero', 'tolerance', 'style', ',', 'clashed']
['on', 'thursday', 'with', 'sunderland', 'no', 'campaigner', 'neil', 'herron', '.', 'mr', 'mallon', 'said', 'an', 'assembly', 'would', 'give', 'local', 'people', 'more', 'of', 'a', 'say']
['over', 'key', 'issues', 'such', 'as', 'transport', 'and', 'crime', '.', 'but', 'mr', 'herron', 'said', 'north', 'east', 'people', 'did', 'not', 'want', 'or', 'need', 'an']
['assembly', '.', 'the', 'pair', 'met', 'on', 'the', 'platform', 'at', 'sunderland', 'station', 'as', 'mr', 'mallon', 'toured', 'the', 'region', 'highlighting', 'claimed', 'improvements', 'to', 'transport']
['if', 'the', 'area', 'gets', 'an', 'assembly

In [7]:

from src.models.rnn_model import RNNMaskedLM, RNNLanguageModel


tokenizer = Tokenizer(vocab=vocab)
rnn = RNNLanguageModel(tokenizer=tokenizer, num_layers=1, bidirectional=False)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [8]:
from torch import nn, optim


loss_fn = nn.CrossEntropyLoss(ignore_index=vocab.pad_token_id)
_optim = optim.Adam(rnn.parameters(), lr=0.001)

rnn.fit(dataset=dataset, loss_fn=loss_fn, optim=_optim, n_epochs=2, batch_size=2)

Step 50: loss = 10.048157691955566
[['in', 'voting', 'reserves', 'integrate', 'revenues', 'the', 'squash', 'redeployed', 'of', 'evening', ',', ',', 'saves', 'in', ',', 'the', '.', 'of', 'of', 'winners', 'of', '6,650', 'it'], ['in', 'century', '.', '.', 'users', 'lopez', '.', 'and', 'the', '.', 'can', ',', '.', '.', 'quarter-finals', 'berlin', 'well-known', 'the', 'the', 'nachison', 'changing', 'strokosch', 'it']]
Step 100: loss = 8.243080139160156
[['of', '.', 'the', 'for', 'the', 'and', 'a', 'and', '``', 'relied', 'relevant', '.', 'to', '.', 'it', 'the', '.', '.', 'the', '.', 'it'], ['of', 'to', '.', '``', 'the', '.', '.', 'the', ':', 'said', 'the', 'the', 'conceals', 'former', '.', 'the', 'the', 'the', '.', '.']]
Step 150: loss = 8.071754455566406
[['of', 'of', 'the', 'of', 'of', '.', 'of', 'that', 'of', 'and', ')', 'family', ',', 'a', 'of', 'the', '.', 'a', 'the', '.', '.', 'the', '.'], ['of', ',', ',', 'the', ',', 'the', 'to', 'to', 'the', 'and', 'the', "'s", ',', 'of', 'the', 'for